In [32]:
import numpy as np
import pandas as pd
%matplotlib qt
import sys
import os
import warnings
from tqdm import tqdm
warnings.filterwarnings("ignore", category=RuntimeWarning)
sys.path.append('../../')
import Utils.Preprocess as ut
project_path = os.path.abspath(os.path.relpath('../../../../', os.getcwd()))
chb_root_path=os.path.join(project_path,'Dataset/CHB-MIT')

constrain_path=os.path.join(project_path,'BilinearNetwork/Data/Constraint/Detection')
seizure_table_original=pd.read_csv(os.path.join(constrain_path,'seizure_ictal_summary.csv'))
exclude_table=pd.read_csv(os.path.join(constrain_path,'exclude_Patient.csv'))['0'].tolist()
save_path=os.path.join(project_path,'BilinearNetwork/Data/PreprocessedData/CHB-MIT/detection2s')

Preictal,ictal

In [33]:
def filter_row(df):
    filtered_rows = []
    for i in range(1, len(df)):
        if df.loc[i, 'File Name'] == df.loc[i-1, 'File Name']:
            duration=(df.loc[i, 'Start Time'] - df.loc[i-1, 'End Time'])
            if  duration < 1800 and not df['File Name'][i].split('_')[0] in exclude_table:
                new=df.loc[i].copy()
                new['duration']=duration
                filtered_rows.append(new)
    return pd.DataFrame(filtered_rows)
# filter_row(seizure_table_original)

In [34]:
def get_preictal_data(EEGTool,seizure_pre_begin,seizure_stop):
    preictal_data=EEGTool.overlap_events(seizure_pre_begin,seizure_stop,overlap=0.5,duration=2).cut_epochs().get_epochs_stft()
    EEGTool.clear()
    return preictal_data
def get_ictal_data(EEGTool,seizure_begin,seizure_stop):
    ictal_data=EEGTool.overlap_events(seizure_begin,seizure_stop,overlap=1.85,duration=2).cut_epochs().get_epochs_stft()
    EEGTool.clear()
    return ictal_data
def get_crossing_data(EEGTool,seizure_begin):
    crossing_data=EEGTool.overlap_events(seizure_begin-2,seizure_begin+2,overlap=1.95,duration=2).cut_epochs().get_epochs_stft()
    EEGTool.clear()
    return crossing_data
def get_file_count(seizure_tb):
    file_count=seizure_tb['File Name'].apply(lambda x: x.split("_")[0])
    return file_count.value_counts()
# file_seizure_counts=get_file_count(seizure_table)
def save_normal_data(preictal_data,ictal_data,patient,i):
    data_X=np.concatenate([preictal_data,ictal_data],axis=0).astype(np.float32)
    data_Y=np.concatenate([np.zeros(preictal_data.shape[0]),np.ones(ictal_data.shape[0])],axis=0).astype(np.float32)
    np.savez(os.path.join(save_path,'{}-normal-{}.npz'.format(patient,i)),X=data_X,y=data_Y)
def save_crossing_data(crossing_data,patient,i):
    data_y=np.linspace(0, 1, crossing_data.shape[0], endpoint=False).astype(np.float32)
    data_X=crossing_data.astype(np.float32)
    np.savez(os.path.join(save_path,'{}-cross-{}.npz'.format(patient,i)),X=data_X,y=data_y)
def save_interictal_data(interictal_data,patient_name):
    data_X=interictal_data.astype(np.float32)
    data_Y=np.zeros(interictal_data.shape[0])
    np.savez(os.path.join(save_path,'{}-interictal.npz'.format(patient_name)),X=data_X,y=data_Y)

In [36]:
patient_name_store=None
current_index=0

# row=seizure_table.iloc[5,:]
for index,row in tqdm(seizure_table_original.iterrows(),total=seizure_table_original.shape[0]):

    file_name=row['File Name']
    patient_name=file_name.split("_")[0]
    if patient_name[-1].isalpha():
        patient_name=patient_name[:-1]
        
    if(patient_name in exclude_table):
        continue

    if(patient_name!=patient_name_store):
        patient_name_store=patient_name
        current_index=0
    
    current_index+=1
    seizure_begin,seizure_stop=row['Start Time'],row['End Time']
    duration=seizure_stop-seizure_begin
    seizure_pre_begin,seizure_pre_stop=seizure_begin-duration*12,seizure_begin
    seizure_pre_begin=seizure_pre_begin if seizure_pre_begin>0 else 0
    seizure_pre_stop=seizure_pre_stop if seizure_pre_stop>0 else 0
    assert seizure_pre_stop>=0
    file_path=os.path.join(chb_root_path,"{}/{}".format(patient_name,file_name))
    raw_T=ut.PreprocessTool(file_path)
    raw_T.do_preprocess()


    preictal_data=get_preictal_data(raw_T,seizure_pre_begin,seizure_pre_stop)


    ictal_data=get_ictal_data(raw_T,seizure_begin,seizure_stop)


    crossing_data=get_crossing_data(raw_T,seizure_begin)
    
    print(preictal_data.shape[0],ictal_data.shape[0],crossing_data.shape[0])

    save_normal_data(preictal_data,ictal_data,patient_name,current_index)
    save_crossing_data(crossing_data,patient_name,current_index)




  0%|          | 0/182 [00:00<?, ?it/s]

Using data from preloaded Raw for 319 events and 513 original time points ...
0 bad epochs dropped
Using data from preloaded Raw for 254 events and 513 original time points ...
0 bad epochs dropped
Using data from preloaded Raw for 41 events and 513 original time points ...
0 bad epochs dropped


  1%|          | 1/182 [00:01<04:00,  1.33s/it]

319 254 41
Using data from preloaded Raw for 215 events and 513 original time points ...
0 bad epochs dropped
Using data from preloaded Raw for 167 events and 513 original time points ...
0 bad epochs dropped
Using data from preloaded Raw for 41 events and 513 original time points ...
0 bad epochs dropped
215 167 41


  1%|          | 2/182 [00:02<03:40,  1.23s/it]

...

Using data from preloaded Raw for 215 events and 513 original time points ...
0 bad epochs dropped
Using data from preloaded Raw for 167 events and 513 original time points ...
0 bad epochs dropped
Using data from preloaded Raw for 41 events and 513 original time points ...
0 bad epochs dropped


 99%|█████████▉| 181/182 [03:55<00:04,  4.11s/it]

215 167 41
Using data from preloaded Raw for 671 events and 513 original time points ...
0 bad epochs dropped
Using data from preloaded Raw for 547 events and 513 original time points ...
0 bad epochs dropped
Using data from preloaded Raw for 41 events and 513 original time points ...
0 bad epochs dropped


100%|██████████| 182/182 [04:00<00:00,  1.32s/it]

671 547 41


interictal

In [11]:
def get_specific_interictal_file(patient_name):
    prefix=patient_name
    edf_files = [
        "chb01_05.edf",
        "chb02_01.edf",
        "chb03_07.edf",
        "chb04_09.edf",
        "chb05_03.edf",
        "chb06_14.edf",
        "chb07_03.edf",
        "chb08_16.edf",
        "chb09_03.edf",
        "chb10_03.edf",
        "chb11_03.edf",
        "chb14_12.edf",
        "chb17a_05.edf",
        "chb18_20.edf",
        "chb19_09.edf",
        "chb20_04.edf",
        "chb21_06.edf",
        "chb22_05.edf",
        "chb23_19.edf"
    ]
    edf_files=pd.Series(edf_files)
    return edf_files[edf_files.str.startswith(prefix)].iloc[0]



In [13]:
for patient_id in range(1,24):

    patient_name='chb'+str(patient_id).zfill(2)
    try:
        file_name=get_specific_interictal_file(patient_name)
    except:
        continue
    file_path=os.path.join(chb_root_path,"{}/{}".format(patient_name,file_name))
    raw_T=ut.PreprocessTool(file_path)
    data_stft=raw_T.do_preprocess(truncate_time=3599).no_overlap_events(start=0,stop=3599).cut_epochs().get_epochs_stft()
    save_interictal_data(interictal_data=data_stft,patient_name=patient_name)



Using data from preloaded Raw for 719 events and 1281 original time points ...
0 bad epochs dropped
Using data from preloaded Raw for 719 events and 1281 original time points ...
0 bad epochs dropped
Using data from preloaded Raw for 719 events and 1281 original time points ...
0 bad epochs dropped
Using data from preloaded Raw for 719 events and 1281 original time points ...
0 bad epochs dropped
Using data from preloaded Raw for 719 events and 1281 original time points ...
0 bad epochs dropped
